<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Setup" data-toc-modified-id="Setup-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Setup</a></span><ul class="toc-item"><li><span><a href="#Helper-functions" data-toc-modified-id="Helper-functions-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Helper functions</a></span></li></ul></li><li><span><a href="#Import-data" data-toc-modified-id="Import-data-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Import data</a></span></li><li><span><a href="#Effect-estimation" data-toc-modified-id="Effect-estimation-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Effect estimation</a></span><ul class="toc-item"><li><span><a href="#Plot-raw-timeseries" data-toc-modified-id="Plot-raw-timeseries-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Plot raw timeseries</a></span></li><li><span><a href="#ITT-on-treated-panelists" data-toc-modified-id="ITT-on-treated-panelists-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>ITT on treated panelists</a></span></li><li><span><a href="#ITT-on-panelists-who-consume-SSBs" data-toc-modified-id="ITT-on-panelists-who-consume-SSBs-3.3"><span class="toc-item-num">3.3&nbsp;&nbsp;</span>ITT on panelists who consume SSBs</a></span></li></ul></li><li><span><a href="#Cook-County" data-toc-modified-id="Cook-County-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Cook County</a></span><ul class="toc-item"><li><span><a href="#Plot-raw-data" data-toc-modified-id="Plot-raw-data-4.1"><span class="toc-item-num">4.1&nbsp;&nbsp;</span>Plot raw data</a></span></li><li><span><a href="#Effects-on-nutrients" data-toc-modified-id="Effects-on-nutrients-4.2"><span class="toc-item-num">4.2&nbsp;&nbsp;</span>Effects on nutrients</a></span><ul class="toc-item"><li><span><a href="#Baseline-DiD" data-toc-modified-id="Baseline-DiD-4.2.1"><span class="toc-item-num">4.2.1&nbsp;&nbsp;</span>Baseline DiD</a></span></li><li><span><a href="#Matching" data-toc-modified-id="Matching-4.2.2"><span class="toc-item-num">4.2.2&nbsp;&nbsp;</span>Matching</a></span></li><li><span><a href="#Synthetic-control" data-toc-modified-id="Synthetic-control-4.2.3"><span class="toc-item-num">4.2.3&nbsp;&nbsp;</span>Synthetic control</a></span></li></ul></li><li><span><a href="#Effects-by-geography" data-toc-modified-id="Effects-by-geography-4.3"><span class="toc-item-num">4.3&nbsp;&nbsp;</span>Effects by geography</a></span></li></ul></li><li><span><a href="#Additional-plots" data-toc-modified-id="Additional-plots-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Additional plots</a></span><ul class="toc-item"><li><span><a href="#Cook-County-map" data-toc-modified-id="Cook-County-map-5.1"><span class="toc-item-num">5.1&nbsp;&nbsp;</span>Cook County map</a></span></li><li><span><a href="#Google-Trends" data-toc-modified-id="Google-Trends-5.2"><span class="toc-item-num">5.2&nbsp;&nbsp;</span>Google Trends</a></span></li></ul></li></ul></div>

# SSB Tax Analysis
By Zachary A. Goodman

Updated: 2021-04

This script performs the analysis in Goodman & Orchard (2021).

## Setup

In [ ]:
clear all

* set working directory
qui cd "../"

* set global directories
global raw_data ./data/raw_data
global temp_data ./data/temp_data
global gen_data ./data/gen_data
global figures ./tex/figures 
global tables ./tex/tables

* check dependencies
which cem
which reghdfe

* plot style
set scheme s1color

### Helper functions

In [ ]:
capture program drop balance_panel
program balance_panel
    /*
    Keeps panelists who are present in panel for all months.
    */
    bys household_code: gen N = _N
    qui sum N
    keep if N == r(max)
    end

In [ ]:
capture program drop top_code
program top_code
    /* 
    Top codes >99th percentile of `var' at the 99th percentile
     to avoid the overreaching effects of outliers.
    */
    args var
    qui sum `var', detail
    replace `var' = r(p99) if `var' > r(p99)
    end

capture program drop top_code_all
program top_code_all
    /* 
    Top codes all nutrient vars.
    */
    foreach v of varlist calories-transfatgrams {
        capture top_code `v'
    }
    end

In [ ]:
* define functions for each of four main samples

capture program drop get_sea_sf get_cook get_boulder_oak get_philly

program get_sea_sf
    keep if inlist(dma_cd, 819, 807) & inlist(locality, 4, 8, 9)
    end

program get_cook
    keep if dma_cd == 602
    end

program get_boulder_oak
    keep if inlist(dma_cd, 751, 807) & inlist(locality, 4, 3, 6)
    end

program get_philly
    keep if dma_cd == 504
    end

capture program drop get_cook data
program get_cook_data

    * get data
    use $gen_data/panelist_nutrition_month_prepped, clear
    get_cook
    keep if yearmonth // < 2019 
    top_code_all

    * add months_since_treated for control units
    bys yearmonth: ereplace months_since_treat = min(months_since_treat)

    * drop unnecessary vars
    drop fips* dma_cd-seattle imputed*
    end

In [ ]:
* function for exporting plots

capture program drop export_img
program export_img
    args name
    graph export $figures/`name'.pdf, name(`name') replace
    end

## Import data

In [ ]:
use $gen_data/panelist_nutrition_month_prepped.dta, clear
top_code_all

## Effect estimation

### Plot raw timeseries

In [ ]:
* plot trends for nutrients

capture restore
preserve

keep if months_since_treat >=-12 & months_since_treat <= 12
collapse (mean) calories-transfatgrams, by(months_since_treat)

sort months_since_treat

foreach v of varlist calories-transfatgrams {
    twoway line `v' months_since_treat, ///
    title("`v'")
}

restore

In [ ]:
* plot trends for control

capture restore
preserve

keep if locality == 4
collapse (mean) calories-transfatgrams, by(yearmonth)

sort yearmonth

foreach v of varlist calories-transfatgrams {
    twoway line `v' yearmonth, ///
    title("`v'")
}

restore

### ITT on treated panelists

In [ ]:
capture program drop get_itt
program get_itt
    /* Estimates ITT of tax on purchases of `nutr' */
    args nutr
    reghdfe `nutr' tau, absorb(yearmonth household_code) vce(cluster panelist_zipcd)
    end

In [ ]:
capture drop logsugargrams
gen logsugargrams = log(sugargrams)

local lst = "sea_sf cook boulder_oak philly"
capture restore
foreach v in `lst' {
    preserve
    get_`v'
    bys yearmonth: ereplace months_since_treat = min(months_since_treat)
    keep if months_since_treat <= 3
    get_itt logsugargrams
    restore
}

In [ ]:
capture drop logsugargrams
gen logsugargrams = log(sugargrams)

* seattle & sf (Jan 2018)
reghdfe logsugargrams tau if inlist(dma_cd, 819, 807) & inlist(locality, 4, 8, 9), ///
    absorb(yearmonth household_code) vce(cluster panelist_zipcd)

* cook (end of 2017)
reghdfe logsugargrams tau if dma_cd == 602, absorb(yearmonth household_code) vce(cluster panelist_zipcd)

* boulder & oakland (mid 2017)
reghdfe logsugargrams tau if inlist(dma_cd, 751, 807) & inlist(locality, 4, 3, 6), ///
    absorb(yearmonth household_code) vce(cluster panelist_zipcd)

* philly (start of 2017)
reghdfe logsugargrams tau if dma_cd == 504 & inlist(locality, 4, 7), ///
    absorb(yearmonth household_code) vce(cluster panelist_zipcd)

### ITT on panelists who consume SSBs

In [ ]:
* TODO - need indicator for SSB consumption

In [ ]:
* Triple diff?

## Cook County

Here we look at Cook Co. by itself given its larger geographical coverage and that it repealed the tax.

In [ ]:
get_cook_data

### Plot raw data

In [ ]:
* Plot function

capture program drop plot_cook
program plot_cook
    /*
    Plots time series of raw data for `var'
    for Cook County vs control.
    */
    args var title plotname ytitle
    twoway (line `var' months_since_treat if evertreated, lcol("cranberry")) ///
        (line `var' months_since_treat if !evertreated, lcol("navy")), ///
        xline(0, lpattern(dash)) xline(3, lpattern(dash)) ///
        title("`title'") ///
        ytitle("`ytitle'") ///
        xtitle("") ///
        name(`plotname', replace) ///
        legend(off) // legend(order(1 "Treatment" 2 "Control")) ///

    end

In [ ]:
* Panelist behaviors

capture restore
preserve

* percent vars
gen p_storebrand = items_storebrand / items_scanned
gen p_food = items_food / items_scanned
gen p_coupon = items_coupons / items_scanned
gen p_deals = items_deals / items_scanned

local vars = "trips total_spent items_deals-coupons_amount_saved p_*"
local titles = "Trips per"
local nvars : word count `vars'

collapse (mean) `vars', by(evertreated months_since_treat)

plot_cook trips "Trips per month" "trips" "Trips/month"
plot_cook total_spent "Expenditures per month" "total_spent" "Dollars/month"
plot_cook final_price_paid "Expenditures scanned per month" "total_scan" "Dollars/month"
plot_cook final_price_paid_food "Food expenditures scanned per month" "total_scan_food" "Dollars/month"
plot_cook items_scanned "Items scanned per month" "items_scan" "Items/month"
plot_cook p_food "Percent scanned items food" "p_food" "Percent"
plot_cook p_storebrand "Percent scanned items storebrand" "p_storebrand" "Percent"
plot_cook p_coupon "Percent items coupon used" "p_coupon" "Percent"
plot_cook p_deals "Percent items with deals" "p_deals" "Percent"

graph combine trips total_spent total_scan total_scan_food ///
    items_scan p_food p_storebrand p_coupon p_deals, ///
    row(3) name(panelist_behav, replace) altshrink iscale(1.2) ysize(5) xsize(6)

* export
export_img panelist_behav


* alt method: iterate over two list
// forval i=1/`nvars' {
//     local thisvar : word `i' of `vars'
//     local thistitle : word `i' of `titles'
//     plot_cook `thisvar' `thistitle'
// }

restore

In [ ]:
* Nutrients

capture restore
preserve

* percent vars
gen p_sugar = (sugargrams * 4) / calories

local vars = "calories-sugargrams p_sugar"

collapse (mean) `vars', by(evertreated months_since_treat)

plot_cook calories "Calories per month" "calories" "kcal/month"
plot_cook sugargrams "Sugars per month" "sugar" "g/month"
plot_cook p_sugar "Percent calories from sugar" "p_sugar" "Percent"
plot_cook fatgrams "Fats per month" "fats" "g/month"
plot_cook satfatgrams "Saturated fats per month" "satfats" "g/month"
plot_cook carbsgrams "Carbohydrates per month" "carbs" "g/month"
plot_cook fibergrams "Fiber per month" "fiber"  "g/month"
plot_cook proteingrams "Protein per month" "protein" "g/month"
plot_cook sodiumgrams "Sodium per month" "sodium" "g/month"

graph combine calories sugar p_sugar fats ///
    satfats carbs fiber protein sodium, ///
    row(3) name(panelist_nutr, replace) altshrink iscale(1.2) ysize(5) xsize(6)

* export
export_img panelist_nutr


restore

### Effects on nutrients

#### Baseline DiD

In [ ]:
* Define function

capture program drop reg_nutr
program reg_nutr
    args nutr nutrname

    * gen months var > 0
    capture drop temp_months
    qui sum months_since_treat
    local min_months = r(min)
    local base = -2 - `min_months'
    gen temp_months = months_since_treat - `min_months'

    * gen treatment var if missing
    capture gen treated = locality_str == "cook"

    * run regression
    capture drop log_`nutr'
    gen log_`nutr' = log(`nutr')
    reghdfe log_`nutr' treated c.treated#ib`base'.temp_months, ///
        absorb(household_code temp_months) vce(cluster panelist_zipcd)
    drop temp_months log_`nutr'

    * coeffplot labels
    local labels = ""
    local keeplist = ""
    forvalues i = 7/34 {
        local j = `i' + `min_months'
        local labels = "`labels'" + `"`i'.* = `j' "'
        local keeplist = "`keeplist'" + `"`i'.* "'
    }

    * coeffplot
    local title = proper("`nutrname'")
    local start_policy = 12.5
    local end_policy = `start_policy' + 4
    coefplot, mcol("navy") ciopts(lcol("navy")) keep(`keeplist') vertical ///
        yline(0, lcol(black)) xline(`start_policy', lpattern(dash)) ///
        xline(`end_policy', lpattern(dash)) ///
        baselevels omitted nolabel coeflabels(`labels') ///
        xtitle("Months since tax began") ///
        ytitle("Percent change in `nutrname' purchased") ///
        title("`title'") ///
        ylabel(-0.2(0.1)0.2) ///
        name(coefplot_cook_`nutr', replace)

    end

In [ ]:
reg_nutr sugargrams "sugar"
reg_nutr calories "calories"
reg_nutr fatgrams "fat"
reg_nutr carbsgrams "carbohydrates"
reg_nutr proteingrams "protein"
reg_nutr fibergrams "fiber"

graph combine coefplot_cook_calories coefplot_cook_sugargrams ///
    coefplot_cook_carbsgrams coefplot_cook_fibergrams ///
    coefplot_cook_fatgrams coefplot_cook_proteingrams, ///
    row(3) name(coefplot_cook_combo, replace) ///
    altshrink iscale(1.2) ysize(5) xsize(6)

export_img coefplot_cook_combo

In [ ]:
* TODO sugar from SSBs
* TODO sugar by store location
* TODO sugar by those who consume SSBs

#### Matching

In [ ]:
* merge demographic information

merge m:1 household_code year using $gen_data/panelist_demographics.dta, keep(3) nogen

In [ ]:
tab hh_emp_male, nolab

In [ ]:
* get matches via CEM

cem hh_race (1 2 3 4) ///
    hh_size (1 2 3 4 5 6) ///
    hh_income (1 2 3 4) ///
    hh_age (1 2 3 4) ///
    hh_educ (1 2 3 4) ///
    hh_child (0 1) ///
    hh_emp_male (0 1 2 3 9) ///
    hh_emp_female (0 1 2 3 9) if year == 2017, tr(evertreated)

In [ ]:
* estimate trends with matched subset

preserve
bys household_code: ereplace cem_matched = min(cem_matched)
keep if cem_matched

reg_nutr sugargrams "sugar"

restore

#### Synthetic control

In [ ]:
get_cook_data
keep if yearmonth < 2019

* aggregate to zip level
collapse (mean) sugargrams total_spent, by(panelist_zipcd yearmonth months_since_treat evertreated tau)

* balance panel
bys panelist_zipcd: keep if _N == 36

In [ ]:
* aggregate treated units

bys evertreated yearmonth: ereplace sugargrams = mean(sugargrams) if evertreated
bys evertreated yearmonth: ereplace total_spent = mean(total_spent) if evertreated
by evertreated yearmonth: keep if _n == 1 | !evertreated
replace panelist_zipcd = 1 if evertreated
// tab yearmonth evertreated


* set panel 
replace months_since_treat = months_since_treat + 19
tsset panelist_zipcd months_since_treat

In [ ]:
export delimited $temp_data/synth_prepped.csv

In [ ]:
* run synthetic control
synth sugargrams total_spent sugargrams, trunit(1) trperiod(19) ///
    figure keep($gen_data/synth.dta) replace

In [ ]:
use $gen_data/synth.dta, clear

In [ ]:
sum

### Effects by geography

In [ ]:
* restore data
get_cook_data
qui do ./jupyter/cook_zip_layers.do
tab layer

In [ ]:
* estimate effects as function of layer

capture restore
preserve

* restrict years
* Why? Isolate effect of tax during tax from effect post removal
keep if yearmonth < 2017.916 // 2019

* prep
gen logsugar = log(sugargrams)

* gen treatment var if missing
capture gen treated = locality_str == "cook"
bys yearmonth: ereplace tau = max(tau) if layer > 0

* run regression
reghdfe logsugar c.tau#i.layer, ///
    absorb(household_code yearmonth) vce(cluster panelist_zipcd)

restore

## Additional plots

### Cook County map

In [ ]:
* Run shp2dta to get Stata recognizable data

shp2dta using $raw_data/maps/zip.shp, ///
    database($raw_data/maps/zipdb) ///
    coordinates($raw_data/maps/zipcoord) ///
    genid(zipid) replace

shp2dta using $raw_data/maps/county.shp, ///
    database($raw_data/maps/cntydb) ///
    coordinates($raw_data/maps/cntycoord) ///
    genid(cntyid) replace

In [ ]:
*** Zip-code layer plot

* Load coordinate data
use $raw_data/maps/zipdb, clear
rename *, lower
gen zip = real(zcta)

* keep only Chicago DMA, and merge border data
merge 1:1 zip using $raw_data/maps/zip_list, keep(3) nogen
merge 1:1 zip using $raw_data/maps/border, keep(3) nogen

* Merge coefficient data
preserve
use $raw_data/maps/coeff_cook_tot_border.dta, clear  // TODO: update this
gen border = real(substr(var, 1, 1))
keep border coef
drop if border == .
rename coef coeff_tot
tempfile temp
qui save `temp'
restore
merge m:1 border using `temp', nogen


** Create Cook Co Boundary
* Load county coordinate data 
preserve

use $raw_data/maps/cntydb, clear
rename *, lower
gen countyfip = real(geoid)
keep if countyfip == 17031
save $raw_data/maps/cntydb2, replace

mergepoly cntyid using $raw_data/maps/cntycoord, ///
    coord($raw_data/maps/cookcoord) replace

restore

* keep relevant lats and longs
gen lat = real(intptlat)
gen lon = real(intptlon)
keep if inrange(lat, 41.2, 42.4)
keep if inrange(lon, -88.4, -87.2)

* Create map using coeff as intensity var
spmap coeff_tot using $raw_data/maps/zipcoord, ///
    id(zipid) ndfcolor(white) clnumber(7) fcolor(RdYlBu) clm(k) ///
    polygon(data($raw_data/maps/cookcoord) fcolor(none) ocolor(yellow) osize(thick)) ///
    name(cookzips, replace) mfcolor(black) ///
    legend(off)
//    title("Cook County layers by zip code")

export_img cookzips

### Google Trends

Examine Google Trends data to have some sense of salience and anticipation effects.

In [ ]:
* import and prep data

import delimited $raw_data/gtrends.csv, varnames(1) clear delimiters(",")
rename sodatax sodatax
gen date = date(week, "MDY")
gen year = year(date)
gen week2 = week(date)
gen month = month(date)

gen yearweek = week2
replace yearweek = yearweek+52 if year == 2017

local passed = week(date("11/10/2016", "MDY"))
local delayed = week(date("6/27/2017", "MDY")) + 52
local start = week(date("8/2/2017", "MDY")) + 52
local delayed2 = week(date("9/13/2017", "MDY")) + 52
local repeal = week(date("10/11/2017", "MDY")) + 52
local end = week(date("12/1/2017", "MDY")) + 52

capture drop label
gen label = ""
replace label = "Passed" if yearweek == `passed'
replace label = "Tax Delayed" if yearweek == `delayed'
replace label = "Start" if yearweek == `start'
replace label = "Vote" if yearweek == `delayed2'
replace label = "Delayed" if yearweek == `delayed2' + 1
replace label = "Repealed" if yearweek == `repeal'
replace label = "End" if yearweek == `end'

capture drop marker*
gen markerx = yearweek - 3
gen markery = sodatax + 4
replace markerx = markerx - 3 if label == "Tax Delayed"
replace markerx = markerx - 1.5 if label == "Start"
replace markerx = markerx - 1 if label == "Repealed"
replace markerx = markerx + 3 if label == "End"
replace markerx = markerx - 1.5 if label == "Delayed"
replace markery = markery + 32 if label == "Delayed"
replace markery = markery + 4 if label == "Vote"
replace markerx = markerx + 1 if label == "Vote"
replace markery = markery - 3 if inlist(label, "Start", "End")


twoway (scatter markery markerx, mlabel(label) mlabcol(black) msize(0) mcol(%0)) ///
    (line sodatax yearweek, lcol(navy) xline(83 100, lpattern(dash))), ///
    xlabel(45 `" "Nov 10" "2016" "' 78 `" "June 27" "2017" "' ///
            93 `" "Oct 11" "2017" "' 100 `" "Dec 1" "2017" "') ///
    legend(off) ytitle("Relative Volume") ///
    name(gtrends, replace)
//    title(`"Relative Weekly Search Volume for "soda tax" "') ///

export_img gtrends